# Image search engine with DocArray

In this notebook, we use [DocArray](https://docarray.jina.ai/) to show how you can set up a simple similar image search. 

## 1. Importing libraries

In [ ]:
from docarray import Document, DocumentArray


## 2. Configuration and setup

We'll set up some basic variables. Feel free to adapt these for your own project!

There is some sample data included in this repository.

The images in the sample dataset are already scaled down to 80x60 to limit the file size.

In [ ]:
MAX_DOCS = 10000
DATA_DIR = '../images/fashion/'
DATA_PATH = f"{DATA_DIR}/**/*.jpg"
QUERY_IMAGE = "../images/fashion/012/0120129025.jpg"


DocArray supports Elasticsearch as a storage backend, one can use it as the follows:

In [ ]:
DocumentArray(
    storage='elasticsearch',
    config={'hosts': 'http://es01:9200',
            'index_name': 'image_search', 'n_dim': 1000},
).summary()


The look-and-feel of a weaviate-powered DocumentArray is exactly the same as an ordinary DocumentArray. All APIs behave the same. We will see that in the sequel.

## 3. Load Data

Now that all is set up, we can start adding the fashion images to the database. 

In [ ]:
docs = DocumentArray.from_files(
    DATA_PATH,
    size=MAX_DOCS,
    storage='elasticsearch',
    config={'hosts': 'http://es01:9200',
            'index_name': 'image_search', 'n_dim': 1000},
)

print(f"{len(docs)} Documents in DocumentArray")


In [ ]:
docs.plot_image_sprites()  # Preview the images


## 4. Preprocess Data

In [ ]:
# Convert to tensor, normalize so they're all similar enough
def preproc(d: Document):
    return (d.load_uri_to_image_tensor()  # load
            # ensure all images right size (dataset image size _should_ be (80, 60))
             .set_image_tensor_shape((80, 60))
             .set_image_tensor_normalization()  # normalize color
             .set_image_tensor_channel_axis(-1, 0))  # switch color axis for the PyTorch model later


In [ ]:
# apply en masse
docs.apply(preproc)


## 5. Embed images

Using GPU is recommended when you want to do image search on large scale. 

In [ ]:
# Use GPU if available
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)


We use the ResNet50 model to embed our images. 

In [ ]:
import torchvision
model = torchvision.models.resnet50()  # load ResNet50


In [ ]:
docs.embed(model, device=device, batch_size=20)


We can remove the tensor of the images to reduce remory usage. This will significantly improve the speed of the `.find()` operation below.

In [ ]:
docs[:, 'tensor'] = None


## 6. Query dataset

Let's query the dataset with one of the images of the dataset. We've stored this in `data/query.jpg`. Feel free to choose any jpg image of clothing as query image.

In [ ]:
query_doc = Document(uri=QUERY_IMAGE)
query_doc.display()


In [ ]:
query_doc = preproc(query_doc)


In [ ]:
# If running on non-gpu machine, change "cuda" to "cpu"
query_doc.embed(model, device=device)


In [ ]:
results = docs.find(query_doc, limit=10)

results[0].summary()

Show the result:

In [ ]:
results[0].plot_image_sprites(image_source="uri")


That's it! You can change the example query image and try out more searches yourself.